In [1]:
from sqlalchemy import func, text, or_, and_
from sqlalchemy.orm.query import Query


In [2]:
from app.core.order import Order, OrderMergedOrder

In [3]:
order_statuses = ["pending", "backorder"]

orders = (Order.query()
    .outerjoin(Order.merged_order)
    .filter(Order.order_date_utc >= text("DATE_SUB(NOW(), INTERVAL 3 MONTH)"))
    .filter(or_(OrderMergedOrder.master == 1, OrderMergedOrder.master.is_(None)))
    .filter(or_(Order.fulfillment_status.is_(None), Order.fulfillment_status.in_(order_statuses)))
    .filter(Order.payment_hold == 0)
    .filter(Order.operator_hold == 0)
    .filter(Order.fraud_hold == 0)
    .filter(~Order.address1.is_(None))
    .filter(~Order.zip.is_(None))
    .filter(~Order.country.is_(None))
    .group_by(Order.account_id, Order.address1, Order.zip, Order.country)
    .having(func.count() > 1)
    .order_by(Order.req_ship_date, Order.id)
)

In [4]:
from sqlalchemy_pretty_sql import pretty_sql

In [5]:
pretty_sql(orders)